In [1]:
import numpy as np
import pandas as pd 



In [2]:
use_cols = ["BUYER_STATE","BUYER_COUNTY", "CALC_BASE_WT_IN_GM", "MME_Conversion_Factor","TRANSACTION_DATE", "DRUG_NAME"]





In [4]:
import pandas as pd
df = pd.read_csv('/Users/dapoadegbile/Desktop/720Python/arcos-fl-statewide-itemized.csv.gz',  compression='gzip', usecols = use_cols,
                chunksize = 1000000,
                error_bad_lines=False)

In [5]:
df

## Shipment data in a dataframe

In [6]:
df1 = pd.DataFrame()
for i in df:
    a = pd.DataFrame(i)
    df1 = df1.append(a)

# for d,i in df.iterrows():
#     print(d)
#     break


In [7]:
df1

,BUYER_STATE,BUYER_COUNTY,DRUG_NAME,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,MME_Conversion_Factor
0,FL,PALM BEACH,OXYCODONE,8182006,3.5860,1.5
1,FL,PALM BEACH,OXYCODONE,11292006,7.1720,1.5
2,FL,PALM BEACH,OXYCODONE,2062007,21.5160,1.5
3,FL,PALM BEACH,OXYCODONE,3012007,21.5160,1.5
4,FL,PALM BEACH,OXYCODONE,4162007,5.3790,1.5
...,...,...,...,...,...,...
15104978,FL,HILLSBOROUGH,OXYCODONE,12042008,268.9500,1.5
15104979,FL,HILLSBOROUGH,HYDROCODONE,8132009,3.0270,1.0
15104980,FL,HILLSBOROUGH,HYDROCODONE,12302009,1.5135,1.0
15104981,FL,HILLSBOROUGH,OXYCODONE,3182010,67.2375,1.5


## Cleaning shipment data frame

In [8]:
df1['TRANSACTION_YEAR'] = df1['TRANSACTION_DATE'].astype(str).str[-4:]


In [9]:
df1['CALC_BASE_WT_IN_MG'] = df1['CALC_BASE_WT_IN_GM'] * 1000

In [10]:
df1['MORPHINE_EQUIV_IN_MG'] = df1['CALC_BASE_WT_IN_MG'] * df1['MME_Conversion_Factor']

final_shipments_FL = df1[["TRANSACTION_YEAR", "BUYER_STATE", "BUYER_COUNTY", "MORPHINE_EQUIV_IN_MG"]]
final_shipments_FL



,TRANSACTION_YEAR,BUYER_STATE,BUYER_COUNTY,MORPHINE_EQUIV_IN_MG
0,2006,FL,PALM BEACH,5379.00
1,2006,FL,PALM BEACH,10758.00
2,2007,FL,PALM BEACH,32274.00
3,2007,FL,PALM BEACH,32274.00
4,2007,FL,PALM BEACH,8068.50
...,...,...,...,...
15104978,2008,FL,HILLSBOROUGH,403425.00
15104979,2009,FL,HILLSBOROUGH,3027.00
15104980,2009,FL,HILLSBOROUGH,1513.50
15104981,2010,FL,HILLSBOROUGH,100856.25


In [11]:
final_shipments_FL = final_shipments_FL.rename(columns={"BUYER_COUNTY": "County", "TRANSACTION_YEAR":"Year"})
final_shipments_FL

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG
0,2006,FL,PALM BEACH,5379.00
1,2006,FL,PALM BEACH,10758.00
2,2007,FL,PALM BEACH,32274.00
3,2007,FL,PALM BEACH,32274.00
4,2007,FL,PALM BEACH,8068.50
...,...,...,...,...
15104978,2008,FL,HILLSBOROUGH,403425.00
15104979,2009,FL,HILLSBOROUGH,3027.00
15104980,2009,FL,HILLSBOROUGH,1513.50
15104981,2010,FL,HILLSBOROUGH,100856.25


In [12]:
final_shipments_FL[["Year"]] = final_shipments_FL["Year"].astype(int)

### CLEAN FLORIDA SHIPMENT DATA and CONCAT [unsure if we need to keep this section]

In [13]:
#Load in florida population DAta
fl_pop_pre_policy_03to09 = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_pre_policy_mortality.csv", index_col= False)
fl_pop_post_policy_10to15 = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_post_policy_mortality.csv", index_col= False)


fl_mortality = pd.concat([fl_pop_pre_policy_03to09,fl_pop_post_policy_10to15])



fl_mortality= fl_mortality.drop(['Unnamed: 0'], axis =1)

# make counties upper case for consistency
fl_mortality[['County']] = fl_mortality['County'].str.upper()
fl_mortality



,Year,State,County,Deaths,Population,Mortality Rate
0,2003,FL,ALACHUA COUNTY,11.0,227022.0,0.000048
1,2003,FL,BAY COUNTY,21.0,155044.0,0.000135
2,2003,FL,BREVARD COUNTY,97.0,504847.0,0.000192
3,2003,FL,BROWARD COUNTY,230.0,1707543.0,0.000135
4,2003,FL,CITRUS COUNTY,16.0,126215.0,0.000127
...,...,...,...,...,...,...
233,2009,FL,SARASOTA COUNTY,75.0,377262.0,0.000199
234,2009,FL,SEMINOLE COUNTY,72.0,419788.0,0.000172
235,2009,FL,ST. JOHNS COUNTY,11.0,186281.0,0.000059
236,2009,FL,ST. LUCIE COUNTY,46.0,274344.0,0.000168


In [14]:


fl_pop = fl_mortality.drop(["State", "Deaths", "Mortality Rate"], axis=1)
fl_pop.head()

,Year,County,Population
0,2003,ALACHUA COUNTY,227022.0
1,2003,BAY COUNTY,155044.0
2,2003,BREVARD COUNTY,504847.0
3,2003,BROWARD COUNTY,1707543.0
4,2003,CITRUS COUNTY,126215.0


## MERGE FLORIDA SHIPMENT DATA WITH POPULATION DATA & FIPS CODE (MORTALITY DATA) 

### PIVOTED AWAY FROM USING FIPS CODE


In [15]:
fl_pre_pop = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_pre_population.csv", index_col= False)
fl_post_pop = pd.read_csv("/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_post_population.csv", index_col= False)


fl_population = pd.concat([fl_pre_pop, fl_post_pop])





In [16]:
fl_population= fl_population.drop(['Unnamed: 0'], axis= 1) # drop unnamed column
fl_population[["County"]] = fl_population["County"].str.upper() #make entire column upper case 
fl_population['County'] = fl_population['County'].astype(str).str[:-7] # remove "county" from county name 
fl_population


,Year,County,Population
0,2003,ALACHUA,227022.0
1,2003,BAKER,23555.0
2,2003,BAY,155044.0
3,2003,BRADFORD,27035.0
4,2003,BREVARD,504847.0
...,...,...,...
397,2015,UNION,15265.0
398,2015,VOLUSIA,516730.0
399,2015,WAKULLA,31523.0
400,2015,WALTON,63066.0


In [17]:
fl_population

,Year,County,Population
0,2003,ALACHUA,227022.0
1,2003,BAKER,23555.0
2,2003,BAY,155044.0
3,2003,BRADFORD,27035.0
4,2003,BREVARD,504847.0
...,...,...,...
397,2015,UNION,15265.0
398,2015,VOLUSIA,516730.0
399,2015,WAKULLA,31523.0
400,2015,WALTON,63066.0


In [18]:
# # this merge isn't right but i kind of want to keep it 

# newdf = pd.merge(final_shipments_FL, fl_pop, on= ["County","Year"], how= 'outer', indicator= "True")
# newdf

In [54]:
#merge new data frame
newestdf = final_shipments_FL.merge(fl_population,  how='left')


In [49]:
newestdf

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG,Population
0,2006,FL,PALM BEACH,5379.00,1284489.0
1,2006,FL,PALM BEACH,10758.00,1284489.0
2,2007,FL,PALM BEACH,32274.00,1286586.0
3,2007,FL,PALM BEACH,32274.00,1286586.0
4,2007,FL,PALM BEACH,8068.50,1286586.0
...,...,...,...,...,...
15104978,2008,FL,HILLSBOROUGH,403425.00,1196773.0
15104979,2009,FL,HILLSBOROUGH,3027.00,1214050.0
15104980,2009,FL,HILLSBOROUGH,1513.50,1214050.0
15104981,2010,FL,HILLSBOROUGH,100856.25,1233498.0


In [74]:
#Check to see how many observations there are for same Year state and county 

newestdf.loc[(newestdf['County'] == "PALM BEACH") & (newestdf['Year'] == 2006) ]

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG,Population
0,2006,FL,PALM BEACH,5379.00,1284489.0
1,2006,FL,PALM BEACH,10758.00,1284489.0
98,2006,FL,PALM BEACH,3027.00,1284489.0
112,2006,FL,PALM BEACH,3027.00,1284489.0
120,2006,FL,PALM BEACH,8068.50,1284489.0
...,...,...,...,...,...
15104142,2006,FL,PALM BEACH,272.43,1284489.0
15104143,2006,FL,PALM BEACH,1452.96,1284489.0
15104144,2006,FL,PALM BEACH,726.48,1284489.0
15104161,2006,FL,PALM BEACH,5448.60,1284489.0


In [76]:
#group combine all observances by year state county and add up all morphine equivalents
newestdf1 = newestdf.groupby(['Year', 'BUYER_STATE', "County", "Population"])["MORPHINE_EQUIV_IN_MG"].sum().reset_index()


In [77]:
newestdf1

,Year,BUYER_STATE,County,Population,MORPHINE_EQUIV_IN_MG
0,2006,FL,ALACHUA,239506.0,8.259662e+07
1,2006,FL,BAKER,25571.0,1.190018e+07
2,2006,FL,BAY,165644.0,6.199810e+07
3,2006,FL,BRADFORD,28506.0,1.342483e+07
4,2006,FL,BREVARD,535138.0,2.011862e+08
...,...,...,...,...,...
568,2014,FL,UNION,15258.0,4.865775e+06
569,2014,FL,VOLUSIA,506715.0,2.134613e+08
570,2014,FL,WAKULLA,31416.0,8.808081e+06
571,2014,FL,WALTON,61186.0,2.938213e+07


In [78]:
newestdf1["MORPH_POP_RATIO"] = newestdf1["MORPHINE_EQUIV_IN_MG"] / newestdf["Population"] 

In [79]:
#newestdf = newestdf.groupby(['Year', 'County', 'BUYER_STATE'])
newestdf1

,Year,BUYER_STATE,County,Population,MORPHINE_EQUIV_IN_MG,MORPH_POP_RATIO
0,2006,FL,ALACHUA,239506.0,8.259662e+07,64.303095
1,2006,FL,BAKER,25571.0,1.190018e+07,9.264524
2,2006,FL,BAY,165644.0,6.199810e+07,48.188075
3,2006,FL,BRADFORD,28506.0,1.342483e+07,10.434461
4,2006,FL,BREVARD,535138.0,2.011862e+08,156.372099
...,...,...,...,...,...,...
568,2014,FL,UNION,15258.0,4.865775e+06,15.378069
569,2014,FL,VOLUSIA,506715.0,2.134613e+08,669.648876
570,2014,FL,WAKULLA,31416.0,8.808081e+06,27.631809
571,2014,FL,WALTON,61186.0,2.938213e+07,92.174615


In [80]:
fl_shipment_and_pop = newestdf1

In [81]:
fl_shipment_and_pop.to_csv(r"/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/fl_shipment_and_pop.csv")